# Epitome of Salt: Sunset Show Notes 
## Author: Kevin Brar
We are going to see how we did at rating the DSK cards before they came out on arena. 

## Guests
Josh, Nate

## DataSet
Link: https://www.17lands.com/tier_list/292091ac82554707928df16d509479ef

Last updated: 2024-11-01 07:15

In [5]:
# Define raters and data file
raters = ['Kevin', 'Nate', 'Josh']
data = 'DSK_tier_list.csv'

In [6]:
import pandas as pd

# Function to read rater ratings
def read_ratings(rater):
    return pd.read_csv(f'data/{rater}.csv')

# Initialize eos DataFrame
eos = pd.DataFrame(columns=['Name', 'Color', 'Rarity'])

# Define the tier to numeric value mapping
tier_num = {'G': 0., 'D-': 1., 'D': 2., 'D+': 3., 
            'C-': 4., 'C': 5., 'C+': 6., 'B-': 7., 
            'B': 8., 'B+': 9., 'A-': 10., 'A': 11., 
            'A+': 12., 'F':0.}

# Read ratings for each rater and merge into eos DataFrame
is_first = True
for rater in raters:
    rater_data = read_ratings(rater)
    if is_first:
        eos['Name'] = rater_data['Name']
        eos['Color'] = rater_data['Color']
        eos['Rarity'] = rater_data['Rarity']
        is_first = False
    eos[rater] = rater_data['Rating']
    eos[f'{rater}_num'] = eos[rater].map(tier_num)

# Drop rows with any missing values
eos = eos.dropna().copy()

# Read the master tier list
master_list = pd.read_csv(f'data/{data}')

# Match names with master tier list and add tiers to eos
eos['Tier'] = eos['Name'].map(dict(zip(master_list['Name'], master_list['Tier'])))
eos['Tier_num'] = eos['Tier'].map(tier_num)

# Dropped entries due to insufficient data 
dropped_entries_initial = eos[eos.isna().any(axis=1)]
print(f"Initial entries to be dropped (due to missing data): {len(dropped_entries_initial)}")
print(dropped_entries_initial)

# Drop rows with missing Tier information
eos = eos.dropna().copy()

# Calculate rating errors for each rater
for rater in raters:
    eos[f'{rater}_error'] = abs(eos['Tier_num'] - eos[f'{rater}_num'])

# The final DataFrame
eos.to_csv('data/data.csv', index=False)
eos

Initial entries to be dropped (due to missing data): 4
                               Name Color Rarity Kevin  Kevin_num Nate  \
130                       Pyroclasm     R      U     C        5.0   D+   
219               "Marina Vendrell"     M      R    C-        4.0   D+   
259  Direlect Attic // Widow's Walk     B      C    C+        6.0    C   
266            Monsterous Emergence     G      C     B        8.0   B-   

     Nate_num Josh  Josh_num Tier  Tier_num  
130       3.0    C       5.0   SB       NaN  
219       3.0   D-       1.0  NaN       NaN  
259       5.0   C-       4.0  NaN       NaN  
266       7.0   B-       7.0  NaN       NaN  


,Name,Color,Rarity,Kevin,Kevin_num,Nate,Nate_num,Josh,Josh_num,Tier,Tier_num,Kevin_error,Nate_error,Josh_error
0,Jump Scare,W,C,C+,6.0,C,5.0,C+,6.0,C,5.0,1.0,0.0,1.0
1,Acrobatic Cheerleader,W,C,C,5.0,C-,4.0,C,5.0,C,5.0,0.0,1.0,0.0
2,Fear of Surveillance,W,C,C-,4.0,C,5.0,C+,6.0,C-,4.0,0.0,1.0,2.0
3,Grand Entryway // Elegant Rotunda,W,C,C+,6.0,B-,7.0,C,5.0,C,5.0,1.0,2.0,0.0
4,Cult Healer,W,C,C-,4.0,C,5.0,C-,4.0,C,5.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Violent Urge,R,U,C+,6.0,D+,3.0,C+,6.0,C,5.0,1.0,2.0,1.0
267,Moldering Gym // Weight Room,G,C,C,5.0,C,5.0,D,2.0,C+,6.0,1.0,1.0,4.0
268,Walk-In Closet // Forgotten Cellar,G,M,G,0.0,C-,4.0,G,0.0,D+,3.0,3.0,1.0,3.0
269,Shroudstomper,M,U,C-,4.0,C-,4.0,C-,4.0,C+,6.0,2.0,2.0,2.0


In [7]:
# See who's predictions did the best 
for rater in raters:
    error = eos[f'{rater}_error'].sum()
    print(f'{rater} error: {error}')

Kevin error: 432.0
Nate error: 433.0
Josh error: 413.0


In [8]:
# biggest misses
num = 10
for rater in raters:
    temp = eos.copy()
    temp = temp.sort_values(by=f'{rater}_error', ascending=False)
    print(temp[['Name', 'Tier', rater]][0:num])

                                    Name Tier Kevin
244                     Dissection Tools   A+    D+
66                       The Mindskinner    D     B
202          Smoky Lounge // Misty Salon    B     D
34                       Ghostly Dancers    A     C
172         Greenhouse // Rickety Gazebo   B-     D
216                The Jolly Balloon Man   B-     D
133  Painter's Studio // Defaced Gallery   D+     B
6                           Living Phone   D-    C+
80                 Winter's Intervention   D+     B
111                 Meathook Massacre II   B-     D
                         Name Tier Nate
244          Dissection Tools   A+   C-
30                   Split Up   B+   D+
41                Piranha Fly   D-   C+
147    The Rollercrusher Ride    A   C+
142          Cursed Recording   C+   D-
242              Ghost Vacuum   C+   D-
145             Waltz of Rage   B+   C-
205  Arabella, Abandoned Doll   B+   C-
112  Overlord of the Balemurk    A   C+
113   Valgavoth, Terror Eate

In [9]:
# hottest takes
num = 10
for rater in raters:
    temp = eos.copy()
    other_raters = [x for x in raters if x != rater]
    temp[f'{rater}_hot'] = abs(((temp[[f'{x}_num' for x in other_raters]].sum(axis=1)) / (len(other_raters))) - temp[f'{rater}_num'])
    temp = temp.sort_values(by=f'{rater}_hot', ascending=False)
    top_num_cols = ['Name', 'Tier'] + [f'{x}' for x in raters]
    top_num = temp[top_num_cols].head(num)
    print(f'Top {num} hottest takes for {rater}:')
    print(top_num)

Top 10 hottest takes for Kevin:
                                      Name Tier Kevin Nate Josh
147                 The Rollercrusher Ride    A    B+   C+   D+
35   Dollmaker's Shop // Porcelain Gallery   B+     A    C    B
29                      Enduring Innocence   B+     A    B    C
148          Charred Foyer // Warped Space    B     B   C-   D+
216                  The Jolly Balloon Man   B-     D   B-   C+
112               Overlord of the Balemurk    A     A   C+    B
63                  Duskmourn's Domination    C     B   D+    C
260           Defiled Crypt // Cadaver Lab   C+     B   C-   C-
66                         The Mindskinner    D     B   C-   C-
183                   Kona, Rescue Beastie   C-     B    C   C-
Top 10 hottest takes for Nate:
                                      Name Tier Kevin Nate Josh
37               Overlord of the Mistmoors   A-     A   C+   A+
30                                Split Up   B+    B-   D+    B
35   Dollmaker's Shop // Porcelain Galler

In [10]:
# Calculate optimism or pessimism metric for each rater
for rater in raters:
    total_difference = (eos[f'{rater}_num'] - eos['Tier_num']).sum()
    if total_difference > 0:
        print(f'{rater} tends to be optimistic by {total_difference} points.')
    elif total_difference < 0:
        print(f'{rater} tends to be pessimistic by {-total_difference} points.')
    else:
        print(f'{rater} has no overall bias.')

Kevin tends to be optimistic by 20.0 points.
Nate tends to be pessimistic by 129.0 points.
Josh tends to be pessimistic by 79.0 points.
